In [5]:
import pandas as pd
from dash import Dash, dcc, html
import requests

In [6]:
def esios_api(indicator:int, token:str, start_date:str, end_date:str, headers:dict):
    r = requests.get('https://api.esios.ree.es/indicators/' + str(indicator) + '?start_date=' + start_date + 'Z&end_date=' + end_date + 'Z', headers=headers)
    print(r.status_code)
    
    return r.json()

In [8]:
token = 'b69f49f19668733416d36d75533072cba8ebf0562564dc2efc32ca04a9dc756e'
headers = dict()
headers['Accept'] = 'application/json; application/vnd.esios-api-v1+json'
headers['Content-Type'] = 'application/json'
headers['Host'] = 'api.esios.ree.es'
headers['x-api-key'] = token
headers['Cookie'] = ''

In [9]:
df = pd.DataFrame(esios_api(600, token, '2023-04-01T00:00', '2023-04-19T00:00', headers=headers)['indicator']['values'])
df.datetime = pd.to_datetime(df.datetime, utc = True)
df.head()

200


,value,datetime,datetime_utc,tz_time,geo_id,geo_name
0,50.00,2023-04-01 00:00:00+00:00,2023-04-01T00:00:00Z,2023-04-01T00:00:00.000Z,1,Portugal
1,53.84,2023-04-01 00:00:00+00:00,2023-04-01T00:00:00Z,2023-04-01T00:00:00.000Z,2,Francia
2,50.00,2023-04-01 00:00:00+00:00,2023-04-01T00:00:00Z,2023-04-01T00:00:00.000Z,3,España
3,93.19,2023-04-01 00:00:00+00:00,2023-04-01T00:00:00Z,2023-04-01T00:00:00.000Z,8824,Reino Unido
4,123.19,2023-04-01 00:00:00+00:00,2023-04-01T00:00:00Z,2023-04-01T00:00:00.000Z,8825,Italia


In [10]:
df.drop(columns=['geo_id', 'tz_time', 'datetime_utc'], inplace=True)
df.head()

,value,datetime,geo_name
0,50.00,2023-04-01 00:00:00+00:00,Portugal
1,53.84,2023-04-01 00:00:00+00:00,Francia
2,50.00,2023-04-01 00:00:00+00:00,España
3,93.19,2023-04-01 00:00:00+00:00,Reino Unido
4,123.19,2023-04-01 00:00:00+00:00,Italia


In [22]:
data = (
    df
    .query("geo_name == 'España'")
    # .assign(Date=lambda data: pd.to_datetime(data["Date"], format="%Y-%m-%d"))
    .sort_values(by="datetime")
)

app = Dash(__name__)

app.layout = html.Div(
    children=[
        html.H1(children="Avocado Analytics"),
        html.P(
            children=(
                "Analyze the behavior of avocado prices and the number"
                " of avocados sold in the US between 2015 and 2018"
            ),
        ),
        dcc.Graph(
            figure={
                "data": [
                    {
                        "x": data["datetime"],
                        "y": data["value"],
                        "type": "lines",
                    },
                ],
                "layout": {"title": "Average Price of Avocados"},
            },
        ),
        dcc.Graph(
            figure={
                "data": [
                    {
                        "x": data["datetime"],
                        "y": data["value"],
                        "type": "lines",
                    },
                ],
                "layout": {"title": "Avocados Sold"},
            },
        ),
    ]
)

if __name__ == "__main__":
    app.run(debug=True, port = 9050)

Dash is running on http://127.0.0.1:9050/

 * Serving Flask app '__main__'
 * Debug mode: on


Traceback (most recent call last):
  File "/home/gelattoo/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/gelattoo/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 1040, in launch_instance
    app.initialize(argv)
  File "/home/gelattoo/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/home/gelattoo/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 677, in initialize
    self.init_sockets()
  File "/home/gelattoo/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 316, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/home/gelattoo/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 251, in _bind_socket
    return self._try_bind_socket(s, port)
  File "/home/gelattoo/.local/lib/python3.8/site-packages/ipykernel

SystemExit: 1

In [23]:
%tb

SystemExit: 1